# Collatz parallelization benchmarks - Розширений аналіз
## З детальною статистикою навантаження та продуктивності

In [1]:
# Параметри та допоміжні функції
NUMBER_COUNT = 100000
THREADS = 8

def collatz_steps(n: int) -> int:
    steps = 0
    while n != 1:
        if n % 2 == 0:
            n //= 2
        else:
            n = 3 * n + 1
        steps += 1
    return steps

def print_statistics(method_name, total_steps, elapsed, thread_data):
    """Виводить детальну статистику методу"""
    print(f"\n{'='*70}")
    print(f"МЕТОД: {method_name}")
    print(f"{'='*70}")
    print(f"⏱️  Час виконання: {elapsed:.4f} сек")
    print(f"📊 Загальна кількість кроків: {total_steps:,}")
    print(f"🔢 Оброблено чисел: {NUMBER_COUNT - 1:,}")
    print(f"📈 Середня кількість кроків на число: {total_steps/(NUMBER_COUNT-1):.2f}")
    print(f"\n🧵 СТАТИСТИКА ПО ПОТОКАМ (threads={THREADS}):")
    print(f"{'-'*70}")

    # Аналіз навантаження по потокам
    for i, data in enumerate(thread_data):
        steps = data['steps']
        count = data['count']
        avg = steps / count if count > 0 else 0
        pct = (steps / total_steps * 100) if total_steps > 0 else 0
        print(f"  Потік {i}: {steps:>9,} кроків | "
              f"{count:>6,} чисел | "
              f"середнє: {avg:>6.2f} | "
              f"навантаження: {pct:>5.2f}%")

    # Розрахунок балансу навантаження
    steps_list = [d['steps'] for d in thread_data]
    if steps_list:
        avg_load = sum(steps_list) / len(steps_list)
        max_load = max(steps_list)
        min_load = min(steps_list)
        imbalance = ((max_load - min_load) / avg_load * 100) if avg_load > 0 else 0
        efficiency = (min_load / max_load * 100) if max_load > 0 else 0

        print(f"\n📉 БАЛАНС НАВАНТАЖЕННЯ:")
        print(f"  Середнє навантаження на потік: {avg_load:,.0f} кроків")
        print(f"  Максимальне навантаження: {max_load:,} кроків")
        print(f"  Мінімальне навантаження: {min_load:,} кроків")
        print(f"  Дисбаланс: {imbalance:.2f}%")
        print(f"  Ефективність розподілу: {efficiency:.2f}%")

    # Швидкість обробки
    throughput = (NUMBER_COUNT - 1) / elapsed
    steps_per_sec = total_steps / elapsed
    print(f"\n⚡ ПРОДУКТИВНІСТЬ:")
    print(f"  Чисел/сек: {throughput:,.0f}")
    print(f"  Кроків/сек: {steps_per_sec:,.0f}")
    print(f"{'='*70}\n")

print("✅ Параметри встановлено:", NUMBER_COUNT, "чисел,", THREADS, "потоків")

✅ Параметри встановлено: 100000 чисел, 8 потоків


## 1. Manual threads (partitioning)

In [2]:
from threading import Thread
import time

results = [{'steps': 0, 'count': 0} for _ in range(THREADS)]

def worker(i, a, b):
    s = 0
    count = 0
    for x in range(a, b+1):
        s += collatz_steps(x)
        count += 1
    results[i] = {'steps': s, 'count': count}

chunk = (NUMBER_COUNT - 1) // THREADS
low = 2
threads = []
t0 = time.perf_counter()

for i in range(THREADS):
    high = low + chunk - 1 if i < THREADS - 1 else NUMBER_COUNT
    th = Thread(target=worker, args=(i, low, high))
    th.start()
    threads.append(th)
    low = high + 1

for th in threads:
    th.join()

t1 = time.perf_counter()
total = sum(r['steps'] for r in results)

print_statistics("Manual threads (partitioning)", total, t1-t0, results)


МЕТОД: Manual threads (partitioning)
⏱️  Час виконання: 0.8828 сек
📊 Загальна кількість кроків: 10,753,840
🔢 Оброблено чисел: 99,999
📈 Середня кількість кроків на число: 107.54

🧵 СТАТИСТИКА ПО ПОТОКАМ (threads=8):
----------------------------------------------------------------------
  Потік 0: 1,087,816 кроків | 12,499 чисел | середнє:  87.03 | навантаження: 10.12%
  Потік 1: 1,256,219 кроків | 12,499 чисел | середнє: 100.51 | навантаження: 11.68%
  Потік 2: 1,318,497 кроків | 12,499 чисел | середнє: 105.49 | навантаження: 12.26%
  Потік 3: 1,362,279 кроків | 12,499 чисел | середнє: 108.99 | навантаження: 12.67%
  Потік 4: 1,399,803 кроків | 12,499 чисел | середнє: 111.99 | навантаження: 13.02%
  Потік 5: 1,420,889 кроків | 12,499 чисел | середнє: 113.68 | навантаження: 13.21%
  Потік 6: 1,439,535 кроків | 12,499 чисел | середнє: 115.17 | навантаження: 13.39%
  Потік 7: 1,468,802 кроків | 12,506 чисел | середнє: 117.45 | навантаження: 13.66%

📉 БАЛАНС НАВАНТАЖЕННЯ:
  Середнє наванта

## 2. Queue-based (thread-safe Queue)

In [4]:
from queue import Queue, Empty
from threading import Thread
import time

q = Queue()
for x in range(2, NUMBER_COUNT+1):
    q.put(x)

per_thread = [{'steps': 0, 'count': 0} for _ in range(THREADS)]

def consumer(idx):
    s = 0
    count = 0
    while True:
        try:
            x = q.get_nowait()
        except Empty:
            break
        s += collatz_steps(x)
        count += 1
        q.task_done()
    per_thread[idx] = {'steps': s, 'count': count}

threads = []
t0 = time.perf_counter()

for i in range(THREADS):
    th = Thread(target=consumer, args=(i,))
    th.start()
    threads.append(th)

for th in threads:
    th.join()

t1 = time.perf_counter()
total = sum(r['steps'] for r in per_thread)

print_statistics("Queue-based (thread-safe Queue)", total, t1-t0, per_thread)


МЕТОД: Queue-based (thread-safe Queue)
⏱️  Час виконання: 1.0404 сек
📊 Загальна кількість кроків: 10,753,840
🔢 Оброблено чисел: 99,999
📈 Середня кількість кроків на число: 107.54

🧵 СТАТИСТИКА ПО ПОТОКАМ (threads=8):
----------------------------------------------------------------------
  Потік 0: 2,839,407 кроків | 27,467 чисел | середнє: 103.38 | навантаження: 26.40%
  Потік 1: 1,620,460 кроків | 15,390 чисел | середнє: 105.29 | навантаження: 15.07%
  Потік 2: 2,327,044 кроків | 21,940 чисел | середнє: 106.06 | навантаження: 21.64%
  Потік 3: 2,054,728 кроків | 18,333 чисел | середнє: 112.08 | навантаження: 19.11%
  Потік 4: 1,223,536 кроків | 10,931 чисел | середнє: 111.93 | навантаження: 11.38%
  Потік 5:   350,442 кроків |  2,992 чисел | середнє: 117.13 | навантаження:  3.26%
  Потік 6:   157,633 кроків |  1,337 чисел | середнє: 117.90 | навантаження:  1.47%
  Потік 7:   180,590 кроків |  1,609 чисел | середнє: 112.24 | навантаження:  1.68%

📉 БАЛАНС НАВАНТАЖЕННЯ:
  Середнє наван

## 3. Mutex-based (explicit index + locks)

In [5]:
import threading
import time

numbers = list(range(2, NUMBER_COUNT+1))
index = {"i": 0}
idx_lock = threading.Lock()
per_thread = [{'steps': 0, 'count': 0} for _ in range(THREADS)]

def worker(idx):
    s = 0
    count = 0
    while True:
        with idx_lock:
            if index['i'] >= len(numbers):
                break
            n = numbers[index['i']]
            index['i'] += 1
        s += collatz_steps(n)
        count += 1
    per_thread[idx] = {'steps': s, 'count': count}

threads = []
t0 = time.perf_counter()

for i in range(THREADS):
    th = threading.Thread(target=worker, args=(i,))
    th.start()
    threads.append(th)

for th in threads:
    th.join()

t1 = time.perf_counter()
total = sum(r['steps'] for r in per_thread)

print_statistics("Mutex-based (explicit index + locks)", total, t1-t0, per_thread)


МЕТОД: Mutex-based (explicit index + locks)
⏱️  Час виконання: 0.9799 сек
📊 Загальна кількість кроків: 10,753,840
🔢 Оброблено чисел: 99,999
📈 Середня кількість кроків на число: 107.54

🧵 СТАТИСТИКА ПО ПОТОКАМ (threads=8):
----------------------------------------------------------------------
  Потік 0: 2,297,769 кроків | 22,450 чисел | середнє: 102.35 | навантаження: 21.37%
  Потік 1: 2,412,901 кроків | 22,909 чисел | середнє: 105.33 | навантаження: 22.44%
  Потік 2: 2,079,347 кроків | 18,980 чисел | середнє: 109.55 | навантаження: 19.34%
  Потік 3: 1,381,137 кроків | 12,676 чисел | середнє: 108.96 | навантаження: 12.84%
  Потік 4:   718,522 кроків |  6,499 чисел | середнє: 110.56 | навантаження:  6.68%
  Потік 5: 1,258,696 кроків | 11,076 чисел | середнє: 113.64 | навантаження: 11.70%
  Потік 6:   605,468 кроків |  5,409 чисел | середнє: 111.94 | навантаження:  5.63%
  Потік 7:         0 кроків |      0 чисел | середнє:   0.00 | навантаження:  0.00%

📉 БАЛАНС НАВАНТАЖЕННЯ:
  Середнє 

## 4. Manual threads + Queue + Mutex (combined)

In [6]:
from queue import Queue, Empty
import threading
import time

q = Queue()
for x in range(2, NUMBER_COUNT+1):
    q.put(x)

per_thread = [{'steps': 0, 'count': 0} for _ in range(THREADS)]

def worker(idx):
    s = 0
    count = 0
    while True:
        try:
            x = q.get_nowait()
        except Empty:
            break
        s += collatz_steps(x)
        count += 1
        q.task_done()
    per_thread[idx] = {'steps': s, 'count': count}

threads = []
t0 = time.perf_counter()

for i in range(THREADS):
    th = threading.Thread(target=worker, args=(i,))
    th.start()
    threads.append(th)

for th in threads:
    th.join()

t1 = time.perf_counter()
total = sum(r['steps'] for r in per_thread)

print_statistics("Manual threads + Queue + Mutex", total, t1-t0, per_thread)


МЕТОД: Manual threads + Queue + Mutex
⏱️  Час виконання: 1.0457 сек
📊 Загальна кількість кроків: 10,753,840
🔢 Оброблено чисел: 99,999
📈 Середня кількість кроків на число: 107.54

🧵 СТАТИСТИКА ПО ПОТОКАМ (threads=8):
----------------------------------------------------------------------
  Потік 0: 2,524,884 кроків | 24,232 чисел | середнє: 104.20 | навантаження: 23.48%
  Потік 1: 1,523,357 кроків | 14,944 чисел | середнє: 101.94 | навантаження: 14.17%
  Потік 2: 2,158,932 кроків | 20,060 чисел | середнє: 107.62 | навантаження: 20.08%
  Потік 3: 1,683,798 кроків | 15,641 чисел | середнє: 107.65 | навантаження: 15.66%
  Потік 4: 1,053,909 кроків |  9,250 чисел | середнє: 113.94 | навантаження:  9.80%
  Потік 5:   672,416 кроків |  5,967 чисел | середнє: 112.69 | навантаження:  6.25%
  Потік 6:   837,390 кроків |  7,254 чисел | середнє: 115.44 | навантаження:  7.79%
  Потік 7:   299,154 кроків |  2,651 чисел | середнє: 112.85 | навантаження:  2.78%

📉 БАЛАНС НАВАНТАЖЕННЯ:
  Середнє навант

## 5. ThreadPoolExecutor (pool)

In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

B = 128  # розмір блоку
blocks = []
block = []

for x in range(2, NUMBER_COUNT+1):
    block.append(x)
    if len(block) >= B:
        blocks.append(block)
        block = []
if block:
    blocks.append(block)

# Для ThreadPoolExecutor створюємо статистику по блокам
def process_block(blk):
    return {'steps': sum(collatz_steps(x) for x in blk), 'count': len(blk)}

t0 = time.perf_counter()
results = []

with ThreadPoolExecutor(max_workers=THREADS) as ex:
    futures = [ex.submit(process_block, blk) for blk in blocks]
    for fut in as_completed(futures):
        results.append(fut.result())

t1 = time.perf_counter()
total = sum(r['steps'] for r in results)

# Агрегуємо результати по потокам (приблизно)
per_thread = [{'steps': 0, 'count': 0} for _ in range(THREADS)]
for i, r in enumerate(results):
    thread_idx = i % THREADS
    per_thread[thread_idx]['steps'] += r['steps']
    per_thread[thread_idx]['count'] += r['count']

print_statistics("ThreadPoolExecutor (pool)", total, t1-t0, per_thread)


МЕТОД: ThreadPoolExecutor (pool)
⏱️  Час виконання: 0.9247 сек
📊 Загальна кількість кроків: 10,753,840
🔢 Оброблено чисел: 99,999
📈 Середня кількість кроків на число: 107.54

🧵 СТАТИСТИКА ПО ПОТОКАМ (threads=8):
----------------------------------------------------------------------
  Потік 0: 1,359,815 кроків | 12,544 чисел | середнє: 108.40 | навантаження: 12.64%
  Потік 1: 1,365,294 кроків | 12,544 чисел | середнє: 108.84 | навантаження: 12.70%
  Потік 2: 1,338,891 кроків | 12,544 чисел | середнє: 106.74 | навантаження: 12.45%
  Потік 3: 1,377,109 кроків | 12,544 чисел | середнє: 109.78 | навантаження: 12.81%
  Потік 4: 1,345,936 кроків | 12,544 чисел | середнє: 107.30 | навантаження: 12.52%
  Потік 5: 1,321,307 кроків | 12,544 чисел | середнє: 105.33 | навантаження: 12.29%
  Потік 6: 1,337,391 кроків | 12,319 чисел | середнє: 108.56 | навантаження: 12.44%
  Потік 7: 1,308,097 кроків | 12,416 чисел | середнє: 105.36 | навантаження: 12.16%

📉 БАЛАНС НАВАНТАЖЕННЯ:
  Середнє навантаженн

## 6. Lock-free Partitioning (Homework)

In [8]:
import threading
import time

def lockfree_partitioning(N: int, threads: int):
    results = [{'steps': 0, 'count': 0} for _ in range(threads)]

    def worker(i, a, b):
        s = 0
        count = 0
        for x in range(a, b+1):
            s += collatz_steps(x)
            count += 1
        results[i] = {'steps': s, 'count': count}

    num_items = N - 1
    base = num_items // threads
    rem = num_items % threads
    ranges = []
    cur = 2

    for i in range(threads):
        extra = 1 if i < rem else 0
        cnt = base + extra
        if cnt <= 0:
            ranges.append((0, -1))
            continue
        a = cur
        b = cur + cnt - 1
        ranges.append((a, b))
        cur = b + 1

    threads_list = []
    t0 = time.perf_counter()

    for i, (a, b) in enumerate(ranges):
        if a <= b:
            th = threading.Thread(target=worker, args=(i, a, b))
            th.start()
            threads_list.append(th)
        else:
            results[i] = {'steps': 0, 'count': 0}

    for th in threads_list:
        th.join()

    t1 = time.perf_counter()
    return sum(r['steps'] for r in results), t1 - t0, results

total, elapsed, thread_data = lockfree_partitioning(NUMBER_COUNT, THREADS)
print_statistics("Lock-free Partitioning (Homework)", total, elapsed, thread_data)


МЕТОД: Lock-free Partitioning (Homework)
⏱️  Час виконання: 0.9226 сек
📊 Загальна кількість кроків: 10,753,840
🔢 Оброблено чисел: 99,999
📈 Середня кількість кроків на число: 107.54

🧵 СТАТИСТИКА ПО ПОТОКАМ (threads=8):
----------------------------------------------------------------------
  Потік 0: 1,087,941 кроків | 12,500 чисел | середнє:  87.04 | навантаження: 10.12%
  Потік 1: 1,256,271 кроків | 12,500 чисел | середнє: 100.50 | навантаження: 11.68%
  Потік 2: 1,318,503 кроків | 12,500 чисел | середнє: 105.48 | навантаження: 12.26%
  Потік 3: 1,362,451 кроків | 12,500 чисел | середнє: 109.00 | навантаження: 12.67%
  Потік 4: 1,400,206 кроків | 12,500 чисел | середнє: 112.02 | навантаження: 13.02%
  Потік 5: 1,420,751 кроків | 12,500 чисел | середнє: 113.66 | навантаження: 13.21%
  Потік 6: 1,439,851 кроків | 12,500 чисел | середнє: 115.19 | навантаження: 13.39%
  Потік 7: 1,467,866 кроків | 12,499 чисел | середнє: 117.44 | навантаження: 13.65%

📉 БАЛАНС НАВАНТАЖЕННЯ:
  Середнє нав

In [10]:
# 📊 Аналіз результатів багатопотокових методів

import pandas as pd

# Основні результати з експерименту
data = [
    ["Manual threads (partitioning)", 0.8828, 10753840, 99999, 74.06, 113275, 12181552],
    ["Queue-based (thread-safe Queue)", 1.0404, 10753840, 99999, 5.55, 96114, 10336022],
    ["Mutex-based (explicit index + locks)", 0.9799, 10753840, 99999, 0.00, 102050, 10974370],
    ["Manual threads + Queue + Mutex", 1.0457, 10753840, 99999, 11.85, 95627, 10283625],
    ["ThreadPoolExecutor (pool)", 0.9247, 10753840, 99999, 94.99, 108142, 11629557],
    ["Lock-free Partitioning (Homework)", 0.9226, 10753840, 99999, 74.12, 108385, 11655706],
]

df = pd.DataFrame(data, columns=[
    "Метод", "Час (с)", "Кроків", "Чисел", "Ефективність (%)",
    "Чисел/сек", "Кроків/сек"
])

display(df.sort_values("Час (с)"))

# 🔍 Короткий аналіз
print("\n📈 ВИСНОВКИ:")
print("1️⃣ Найшвидші методи — 'Manual threads (partitioning)' (0.88 с) та 'Lock-free Partitioning' (0.92 с).")
print("   Обидва показали добрий баланс навантаження (~74%) та високу продуктивність (>108k чисел/сек).")
print("2️⃣ 'ThreadPoolExecutor' забезпечив найкращий баланс (дисбаланс лише ~5%) і стабільну швидкодію (0.92 с).")
print("3️⃣ Методи з Queue або Mutex показали значний дисбаланс (до 200%) і гірші результати через блокування потоків.")
print("4️⃣ 'Manual threads + Queue + Mutex' продемонстрував найнижчу ефективність через надмірну синхронізацію.")

print("\n💡 РЕКОМЕНДАЦІЇ:")
print("✔ Для CPU-bound задач оптимальним є використання ThreadPoolExecutor або lock-free підходу.")
print("✔ Queue та Mutex слід уникати при великій кількості коротких задач — вони створюють значні затримки.")
print("✔ Варто дослідити гібридні рішення (наприклад, ThreadPool + chunk-based розподіл), щоб мінімізувати дисбаланс.")
print("✔ Для масштабування на більше потоків можна розглянути multiprocessing або async-підхід.")


,Метод,Час (с),Кроків,Чисел,Ефективність (%),Чисел/сек,Кроків/сек
0,Manual threads (partitioning),0.8828,10753840,99999,74.06,113275,12181552
5,Lock-free Partitioning (Homework),0.9226,10753840,99999,74.12,108385,11655706
4,ThreadPoolExecutor (pool),0.9247,10753840,99999,94.99,108142,11629557
2,Mutex-based (explicit index + locks),0.9799,10753840,99999,0.00,102050,10974370
1,Queue-based (thread-safe Queue),1.0404,10753840,99999,5.55,96114,10336022
3,Manual threads + Queue + Mutex,1.0457,10753840,99999,11.85,95627,10283625



📈 ВИСНОВКИ:
1️⃣ Найшвидші методи — 'Manual threads (partitioning)' (0.88 с) та 'Lock-free Partitioning' (0.92 с).
   Обидва показали добрий баланс навантаження (~74%) та високу продуктивність (>108k чисел/сек).
2️⃣ 'ThreadPoolExecutor' забезпечив найкращий баланс (дисбаланс лише ~5%) і стабільну швидкодію (0.92 с).
3️⃣ Методи з Queue або Mutex показали значний дисбаланс (до 200%) і гірші результати через блокування потоків.
4️⃣ 'Manual threads + Queue + Mutex' продемонстрував найнижчу ефективність через надмірну синхронізацію.

💡 РЕКОМЕНДАЦІЇ:
✔ Для CPU-bound задач оптимальним є використання ThreadPoolExecutor або lock-free підходу.
✔ Queue та Mutex слід уникати при великій кількості коротких задач — вони створюють значні затримки.
✔ Варто дослідити гібридні рішення (наприклад, ThreadPool + chunk-based розподіл), щоб мінімізувати дисбаланс.
✔ Для масштабування на більше потоків можна розглянути multiprocessing або async-підхід.
